# DEPI Final Data Science Project 
## Project: Sales Forecasting and Optimization


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller

In [3]:
from pathlib import Path
# Create visualization directory
viz_dir = Path('visualizations')
viz_dir.mkdir(exist_ok=True)

def save_plotly_chart(fig, filename):
    """Save Plotly chart externally (not embedded)"""
    fig.write_html(viz_dir / filename)
    print(f"✅ Chart saved: {filename}")

def save_plt_chart(title, dpi=100):
    """Save matplotlib chart externally"""
    import matplotlib.pyplot as plt
    plt.savefig(viz_dir / f'{title}.png', dpi=dpi, bbox_inches='tight')
    plt.close()
    print(f"✅ Figure saved: {title}.png")

In [4]:
from notebook_optimization import NotebookOptimizer

optimizer = NotebookOptimizer("Final project.ipynb")
#optimizer.strip_outputs()
#optimizer.clean_metadata()
#optimizer.save_clean_version()


##### displaying some info 

In [5]:
train = pd.read_csv('data/train.csv', parse_dates=['Date'], low_memory=False)
store = pd.read_csv('data/store.csv')
print(f"Train dataset loaded: {len(train):,} rows, {len(train.columns)} columns")
print(f"Store dataset loaded: {len(store):,} stores, {len(store.columns)} columns")
print(f"Date range: {train['Date'].min()} to {train['Date'].max()}")

Train dataset loaded: 1,017,209 rows, 9 columns
Store dataset loaded: 1,115 stores, 10 columns
Date range: 2013-01-01 00:00:00 to 2015-07-31 00:00:00


##### preprocessing step to all add store related attributes in the training data (for max efficiency)

In [6]:
df = train.merge(store, on='Store', how='left')
print(f" Merged dataset: {len(df):,} rows, {len(df.columns)} columns")


 Merged dataset: 1,017,209 rows, 18 columns


In [7]:
print(f"Total Records: {len(df):,}")
print(f"Date Range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Number of Stores: {df['Store'].nunique()}")
print(f"Time Period: {(df['Date'].max() - df['Date'].min()).days} days")
print(df.dtypes)

Total Records: 1,017,209
Date Range: 2013-01-01 00:00:00 to 2015-07-31 00:00:00
Number of Stores: 1115
Time Period: 941 days
Store                                 int64
DayOfWeek                             int64
Date                         datetime64[ns]
Sales                                 int64
Customers                             int64
Open                                  int64
Promo                                 int64
StateHoliday                         object
SchoolHoliday                         int64
StoreType                            object
Assortment                           object
CompetitionDistance                 float64
CompetitionOpenSinceMonth           float64
CompetitionOpenSinceYear            float64
Promo2                                int64
Promo2SinceWeek                     float64
Promo2SinceYear                     float64
PromoInterval                        object
dtype: object


In [8]:
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
print(pd.DataFrame({ 'Missing_Count': missing[missing > 0], 'Percentage': missing_pct[missing > 0] }))

                           Missing_Count  Percentage
CompetitionDistance                 2642    0.259730
CompetitionOpenSinceMonth         323348   31.787764
CompetitionOpenSinceYear          323348   31.787764
Promo2SinceWeek                   508031   49.943620
Promo2SinceYear                   508031   49.943620
PromoInterval                     508031   49.943620


In [9]:
print(df[['Sales', 'Customers', 'CompetitionDistance']].describe())

              Sales     Customers  CompetitionDistance
count  1.017209e+06  1.017209e+06         1.014567e+06
mean   5.773819e+03  6.331459e+02         5.430086e+03
std    3.849926e+03  4.644117e+02         7.715324e+03
min    0.000000e+00  0.000000e+00         2.000000e+01
25%    3.727000e+03  4.050000e+02         7.100000e+02
50%    5.744000e+03  6.090000e+02         2.330000e+03
75%    7.856000e+03  8.370000e+02         6.890000e+03
max    4.155100e+04  7.388000e+03         7.586000e+04


In [10]:
print(f"Duplicates: {df.duplicated().sum()}")

Duplicates: 0


##### outlier detection for "Sales" column

In [11]:
Q1 = df['Sales'].quantile(0.25)
Q3 = df['Sales'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR # we use 3*IQR for stricter outlier detection
upper_bound = Q3 + 3 * IQR
outliers = df[(df['Sales'] < lower_bound) | (df['Sales'] > upper_bound)]
print(f"Sales outliers: {len(outliers):,} ({len(outliers)/len(df)*100:.2f}%)")

Sales outliers: 3,772 (0.37%)


In [12]:
df.shape

(1017209, 18)

In [13]:
df = df[(df['Sales'] >= lower_bound) & (df['Sales'] <= upper_bound)].copy()

print(f"Cleaned dataset shape: {df.shape}")

Cleaned dataset shape: (1013437, 18)


In [14]:
fig = make_subplots(rows=3, cols=2,
    subplot_titles=(
        'Daily Sales Trend', 'Sales Distribution', 'Sales by Day of Week',
        'Promo vs No Promo', 'Sales by Store Type', 'Competition Distance Impact'
    ),
    specs=[
        [{'type': 'scatter'}, {'type': 'histogram'}],
        [{'type': 'bar'}, {'type': 'box'}],
        [{'type': 'box'}, {'type': 'scatter'}]
    ]
)
daily_sales = df.groupby('Date')['Sales'].mean().reset_index()
fig.add_trace(go.Scatter(x=daily_sales['Date'], y=daily_sales['Sales'], mode='markers', name='Avg Daily Sales'), row=1, col=1)
fig.add_trace(go.Histogram(x=df['Sales'], nbinsx=50, name='Sales Dist'), row=1, col=2)
dow_sales = df.groupby('DayOfWeek')['Sales'].mean()
fig.add_trace(go.Bar(x=['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], y=dow_sales.values, name='DoW Sales'), row=2, col=1)
fig.add_trace(go.Box(x=df['Promo'].map({0: 'No Promo', 1: 'Promo'}), y=df['Sales'], name='Promo'), row=2, col=2)
if 'StoreType' in df.columns:
    fig.add_trace(go.Box(x=df['StoreType'], y=df['Sales'], name='Store Type'), row=3, col=1)
if 'CompetitionDistance' in df.columns:
    sample = df.sample(min(5000, len(df)))
    fig.add_trace(go.Scatter(x=sample['CompetitionDistance'], y=sample['Sales'], mode='markers', marker=dict(size=3, opacity=0.5)), row=3, col=2)
fig.update_layout(height=1200,showlegend=False,title_text="Rossmann Sales - Initial EDA Dashboard")
save_plotly_chart(fig, 'Rossmann Sales - Initial EDA Dashboard.html')  # ✅ Saves externally!


✅ Chart saved: Rossmann Sales - Initial EDA Dashboard.html


##### Data Preprocessing, cleaning and Feature Engineering


In [15]:
df= df.drop_duplicates()
df['CompetitionDistance'].fillna(df['CompetitionDistance'].median(), inplace=True)
df['CompetitionOpenSinceMonth'].fillna(0, inplace=True)
df['CompetitionOpenSinceYear'].fillna(0, inplace=True)


In [16]:
df['Promo2SinceWeek'].fillna(0, inplace=True)
df['Promo2SinceYear'].fillna(0, inplace=True)
df['PromoInterval'].fillna('None', inplace=True)

In [17]:
Q1 = df['Sales'].quantile(0.25)
Q3 = df['Sales'].quantile(0.75)
IQR = Q3 - Q1
df['Is_Outlier'] = ((df['Sales'] < (Q1 - 3 * IQR)) | (df['Sales'] > (Q3 + 3 * IQR))).astype(int)

In [18]:
df.isna().sum()


Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
Is_Outlier                   0
dtype: int64

##### preparing the a cleaned dataset for feature engineering

In [19]:
df_feat = df.copy()
df_feat = df_feat.sort_values(['Store', 'Date']).reset_index(drop=True)

##### extracts calendar components from date column and store them in new columns

In [20]:
df_feat['Year'] = df_feat['Date'].dt.year
df_feat['Month'] = df_feat['Date'].dt.month
df_feat['Day'] = df_feat['Date'].dt.day
df_feat['WeekOfYear'] = df_feat['Date'].dt.isocalendar().week
df_feat['Quarter'] = df_feat['Date'].dt.quarter

In [21]:
df_feat['IsWeekend'] = (df_feat['DayOfWeek'] >= 6).astype(int) # Saturday=6, Sunday=7, monday=1
df_feat['IsMonthStart'] = df_feat['Date'].dt.is_month_start.astype(int)
df_feat['IsMonthEnd'] = df_feat['Date'].dt.is_month_end.astype(int)

##### this will help machine learning understand cyclical time patterns
##### (eg: that december comes january and after saturday comes sunday)

In [22]:
df_feat['Month_sin'] = np.sin(2 * np.pi * df_feat['Month'] / 12)
df_feat['Month_cos'] = np.cos(2 * np.pi * df_feat['Month'] / 12)
df_feat['DayOfWeek_sin'] = np.sin(2 * np.pi * df_feat['DayOfWeek'] / 7)
df_feat['DayOfWeek_cos'] = np.cos(2 * np.pi * df_feat['DayOfWeek'] / 7)

In [23]:
months = np.arange(1, 13)
month_sin = np.sin(2 * np.pi * months / 12)
month_cos = np.cos(2 * np.pi * months / 12)


fig.add_trace(go.Scatter(
    x=month_cos,
    y=month_sin,
    mode='markers+text',
    text=[f'Month {m}' for m in months],
    textposition='top center',
    marker=dict(size=12, color=months, colorscale='Viridis')
))
# circle outline
theta = np.linspace(0, 2 * np.pi, 100)
fig.add_trace(go.Scatter(
    x=np.cos(theta),
    y=np.sin(theta),
    mode='lines',
    line=dict(color='lightgray', dash='dot'),
    showlegend=False
))
fig.update_layout(
    title='Cyclical Encoding of Months (sin/cos)',
    xaxis_title='cos(2π * month / 12)',
    yaxis_title='sin(2π * month / 12)',
    width=600, height=600,
    xaxis=dict(scaleanchor='y', scaleratio=1),
    yaxis=dict(showgrid=False)
)
save_plotly_chart(fig, 'Cyclical Encoding of Months (sin_cos).html')  # ✅ Saves externally!

✅ Chart saved: Cyclical Encoding of Months (sin_cos).html


##### represent past values of “Sales” and “Customers” for each store

In [24]:
for lag in [1, 7, 14, 30]:#lag periods:1 day, 7 days, 14 days, and 30 days
    df_feat[f'Sales_Lag_{lag}'] = df_feat.groupby('Store')['Sales'].shift(lag)
    df_feat[f'Customers_Lag_{lag}'] = df_feat.groupby('Store')['Customers'].shift(lag)


In [25]:
for window in [7, 14, 30]:
    df_feat[f'Sales_Rolling_Mean_{window}'] = df_feat.groupby('Store')['Sales'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
    df_feat[f'Sales_Rolling_Std_{window}'] = df_feat.groupby('Store')['Sales'].transform(lambda x: x.rolling(window=window, min_periods=1).std())
df_feat['SalesPerCustomer'] = df_feat['Sales'] / df_feat['Customers']

##### These features help a machine learning model recognize patterns over time and Shows the recent average performance of a store

In [26]:
df_feat[f'Sales_Rolling_Mean_{window}'] = df_feat.groupby('Store')['Sales'].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
df_feat[f'Sales_Rolling_Std_{window}'] = df_feat.groupby('Store')['Sales'].transform(lambda x: x.rolling(window=window, min_periods=1).std())
df_feat['SalesPerCustomer'] = df_feat['Sales'] / df_feat['Customers']
df_feat['SalesPerCustomer'].replace([np.inf, -np.inf], 0, inplace=True) # if record has no customers, set SalesPerCustomer to 0

##### this block creates a new feature called CompetitionMonthsOpen, which measures how many months a store has had competition nearby up to the current date (based on the record’s year and month)

In [27]:
if 'CompetitionOpenSinceYear' in df_feat.columns:
        #calculates how many months have passed since a store’s competition started
    df_feat['CompetitionMonthsOpen'] = 12 * (df_feat['Year'] - df_feat['CompetitionOpenSinceYear']) + (df_feat['Month'] - df_feat['CompetitionOpenSinceMonth'])
    df_feat['CompetitionMonthsOpen'] = df_feat['CompetitionMonthsOpen'].clip(lower=0)

##### it measures how long a store’s continuous promotion program (“Promo2”) has been running

In [28]:
if 'Promo2SinceYear' in df_feat.columns:
    df_feat['Promo2Weeks'] = 52 * (df_feat['Year'] - df_feat['Promo2SinceYear']) + (df_feat['WeekOfYear'] - df_feat['Promo2SinceWeek'])
    df_feat['Promo2Weeks'] = df_feat['Promo2Weeks'].clip(lower=0)

##### finally ensuring that the final dataset has no missing values before passing it into a machine learning model

In [29]:
df_feat = df_feat.dropna()  
print(f"Feature engineered shape: {df_feat.shape}")


Feature engineered shape: (812824, 48)


## Statistical Analysis

##### The Augmented Dickey-Fuller test for stationarity on the daily aggregated Rossmann sales data yields an ADF statistic of -70.56 and a p-value below 0.0001. We conclusively reject the null hypothesis of a unit root, indicating that the time series is stationary and suitable for time series modeling without further differencing."
- we’re ready to proceed to training time series models (AR, ARIMA, SARIMAX, Prophet, etc.)

In [30]:
subset = df_feat['Sales'].sample(50000, random_state=1)
adf_result = adfuller(subset.dropna())
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")


ADF Statistic: -129.7046
p-value: 0.0000


In [31]:
numeric_cols = df_feat.select_dtypes(include=[np.number]).columns.tolist()
correlation_matrix = df_feat[numeric_cols].corr()
sales_corr = correlation_matrix['Sales'].abs().sort_values(ascending=False)
print("Top 15 features correlated with Sales:")
print(sales_corr.head(48)[1:]) 

Top 15 features correlated with Sales:
Customers                    0.804878
Sales_Rolling_Mean_7         0.796451
Sales_Rolling_Mean_30        0.777618
Sales_Rolling_Mean_14        0.772597
Sales_Rolling_Std_7          0.733644
Sales_Lag_14                 0.701175
Sales_Rolling_Std_30         0.684401
Sales_Rolling_Std_14         0.675307
Customers_Lag_14             0.614343
Customers_Lag_7              0.520310
Sales_Lag_7                  0.502012
Promo                        0.380655
Sales_Lag_30                 0.373012
Customers_Lag_30             0.371565
Sales_Lag_1                  0.350079
Customers_Lag_1              0.315951
SalesPerCustomer             0.205977
DayOfWeek                    0.184199
IsWeekend                    0.161697
DayOfWeek_sin                0.130156
Promo2Weeks                  0.119075
Promo2SinceYear              0.119041
Promo2                       0.119016
IsMonthEnd                   0.064703
WeekOfYear                   0.063337
Month      

##### measures how much promotions increase sales on average

In [32]:
promo_impact = df_feat.groupby('Promo')['Sales'].agg(['mean', 'median', 'std'])
promo_impact.index = ['No Promo', 'With Promo']
promo_lift = ((promo_impact.loc['With Promo', 'mean'] / promo_impact.loc['No Promo', 'mean']) - 1) * 100
print(f"Avg sales lift from promotions: {promo_lift:.2f}%")

Avg sales lift from promotions: 37.70%


In [33]:
if 'StoreType' in df_feat.columns:
    print(df_feat.groupby('StoreType')['Sales'].agg(['mean', 'median', 'std', 'count']))
dow_stats = df_feat.groupby('DayOfWeek')['Sales'].agg(['mean', 'median'])
dow_stats.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
print(dow_stats)


                  mean  median          std   count
StoreType                                          
a          6855.150511  6283.0  3054.902223  439569
b          9654.975867  8913.0  4271.941298   14420
c          6916.547880  6421.0  2797.712897  109055
d          6831.921627  6419.0  2494.009973  249780
            mean  median
Mon  8089.966816  7531.0
Tue  7042.201993  6509.0
Wed  6711.440669  6224.0
Thu  6745.887804  6257.0
Fri  7039.527111  6589.0
Sat  5851.409575  5433.0
Sun  7319.967809  6609.0


In [34]:
import plotly.express as px

top_features = correlation_matrix['Sales'].abs().sort_values(ascending=False).head(20).index
corr_subset = correlation_matrix.loc[top_features, top_features]

fig = px.imshow(
    corr_subset,
    text_auto=".2f",
    color_continuous_scale='RdBu_r',
    origin='lower',
    title="Top 20 Features - Correlation Heatmap"
)

save_plotly_chart(fig, 'Top 20 Features - Correlation Heatmap.html')  # ✅ Saves externally!

✅ Chart saved: Top 20 Features - Correlation Heatmap.html


In [35]:
sample_sales = df_feat.groupby('Date')['Sales'].mean().dropna().sample(min(900, len(df_feat)))
# Compute ACF and PACF values (up to 50 lags)
lags = 50
acf_values = acf(sample_sales, nlags=lags)
pacf_values = pacf(sample_sales, nlags=lags)
# Create a 1x2 subplot layout
fig = make_subplots(rows=1, cols=2, subplot_titles=("Autocorrelation Function (ACF)", "Partial Autocorrelation (PACF)"))
# ACF plot
fig.add_trace(
    go.Bar(x=list(range(lags + 1)), y=acf_values, name='ACF', marker_color='skyblue'),
    row=1, col=1
)
# PACF plot
fig.add_trace(
    go.Bar(x=list(range(lags + 1)), y=pacf_values, name='PACF', marker_color='lightgreen'),
    row=1, col=2
)
# Add horizontal zero lines
for i in range(1, 3):
    fig.add_shape(type="line", x0=0, x1=lags, y0=0, y1=0, line=dict(color="black", width=1), row=1, col=i)
# Update layout
fig.update_layout(
    title_text="ACF and PACF Plots (Interactive)",
    showlegend=False,
    height=500,
    width=1000,
    template="plotly_white"
)
save_plotly_chart(fig, 'Interactive ACF and PACF Plots.html')  # ✅ Saves externally!

✅ Chart saved: Interactive ACF and PACF Plots.html


In [36]:
#df_feat.to_csv('data/cleaned_sales_features.csv', index=False)
#print(" Cleaned feature dataset saved to: data/cleaned_sales_features.csv")


## Forecasting Model Development and Optimization Objectives

In [37]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
#how to fix pmdarima installation issues?
#import pmdarima as pm
# machine learning libraries
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
#Deep learning libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical



In [40]:
df_feat.shape

(812824, 48)

In [41]:
daily_sales = df_feat.groupby('Date').agg({
    'Sales': 'sum',
    'Customers': 'sum',
    'Promo': 'mean',
    'SchoolHoliday': 'max',
    'StateHoliday': lambda x: (x != '0').any().astype(int)
}).reset_index()

In [42]:
daily_sales = daily_sales.sort_values('Date')
print(f"📊 Daily aggregated sales: {len(daily_sales)} days")
print(f"📅 Date Range: {daily_sales['Date'].min()} to {daily_sales['Date'].max()}")

📊 Daily aggregated sales: 912 days
📅 Date Range: 2013-01-31 00:00:00 to 2015-07-31 00:00:00


In [49]:
test_weeks = 6
test_size = test_weeks * 7
train_size = len(daily_sales) - test_size

train_data = daily_sales.iloc[:train_size].copy()
test_data = daily_sales.iloc[train_size:].copy()

print(f"📈 Training: {len(train_data)} days ({train_data['Date'].min()} to {train_data['Date'].max()})")
print(f"📊 Testing: {len(test_data)} days ({test_data['Date'].min()} to {test_data['Date'].max()})")
y_true = test_data['Sales'].values


📈 Training: 870 days (2013-01-31 00:00:00 to 2015-06-19 00:00:00)
📊 Testing: 42 days (2015-06-20 00:00:00 to 2015-07-31 00:00:00)


In [53]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_data['Date'], y=train_data['Sales'],
                        mode='lines', name='Training Data', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=test_data['Date'], y=test_data['Sales'],
                        mode='lines', name='Test Data', line=dict(color='red')))
fig.update_layout(title='Train-Test Split - Daily Aggregated Sales', height=400)
save_plotly_chart(fig, '01_train_test_split.html')
results_list = []

✅ Chart saved: 01_train_test_split.html


In [54]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def calculate_metrics(y_true, y_pred, model_name=None):
    """Calculate comprehensive metrics with safe MAPE handling"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    # Safe MAPE (avoid division by zero)
    y_true_safe = np.maximum(y_true, 1)
    mape = np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100
    
    # R² score
    r2 = r2_score(y_true, y_pred)
    
    result = {
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R2': r2
    }
    
    if model_name:
        print(f"✓ {model_name:20s} → RMSE: {rmse:>10,.0f} | MAE: {mae:>10,.0f} | MAPE: {mape:>7.2f}% | R²: {r2:>7.4f}")
    
    return result

print("✅ Metrics function ready!")

✅ Metrics function ready!


In [55]:
print("\n" + "="*70)
print("🎯 BASELINE MODELS (Benchmarks)")
print("="*70)

# Naive forecast (last train value)
naive_forecast = np.repeat(train_data['Sales'].iloc[-1], len(test_data))
naive_metrics = calculate_metrics(y_true, naive_forecast, "Naive (Last Value)")
results_list.append(('Naive', naive_metrics))

# Seasonal Naive (7-day)
seasonal_naive = np.tile(train_data['Sales'].iloc[-7:].values, 
                         (len(test_data) // 7) + 1)[:len(test_data)]
seasonal_metrics = calculate_metrics(y_true, seasonal_naive, "Seasonal Naive (7-day)")
results_list.append(('Seasonal Naive', seasonal_metrics))

# Moving Average (7-day)
ma_forecast = np.repeat(train_data['Sales'].tail(7).mean(), len(test_data))
ma_metrics = calculate_metrics(y_true, ma_forecast, "Moving Average (7-day)")
results_list.append(('Moving Average', ma_metrics))

print(f"\n✓ Baseline RMSE range: {naive_metrics['RMSE']:,.0f} - {seasonal_metrics['RMSE']:,.0f}")
print(f"✓ All other models must beat these baselines!")


🎯 BASELINE MODELS (Benchmarks)
✓ Naive (Last Value)   → RMSE:  3,442,355 | MAE:  2,425,738 | MAPE:  516.39% | R²: -0.3498
✓ Seasonal Naive (7-day) → RMSE:  1,651,649 | MAE:  1,094,949 | MAPE:   16.33% | R²:  0.6893
✓ Moving Average (7-day) → RMSE:  3,088,281 | MAE:  2,148,412 | MAPE:  458.57% | R²: -0.0864

✓ Baseline RMSE range: 3,442,355 - 1,651,649
✓ All other models must beat these baselines!


In [56]:
print("\n" + "="*70)
print("🔄 MODEL 1: SMART ARIMA (Grid Search)")
print("="*70)

best_aic = np.inf
best_order = None
best_arima_model = None

print("Searching ARIMA parameters...")

for p in range(0, 4):
    for d in range(0, 2):
        for q in range(0, 4):
            try:
                model = ARIMA(train_data['Sales'].values, order=(p, d, q))
                result = model.fit()
                if result.aic < best_aic:
                    best_aic = result.aic
                    best_order = (p, d, q)
                    best_arima_model = result
            except:
                continue

print(f"✓ Best ARIMA: {best_order} (AIC: {best_aic:.0f})")

arima_forecast = best_arima_model.forecast(steps=len(test_data))
arima_metrics = calculate_metrics(y_true, arima_forecast, f"ARIMA{best_order}")
results_list.append((f'ARIMA{best_order}', arima_metrics))


🔄 MODEL 1: SMART ARIMA (Grid Search)
Searching ARIMA parameters...
✓ Best ARIMA: (2, 0, 3) (AIC: 28229)
✓ ARIMA(2, 0, 3)       → RMSE:  2,504,064 | MAE:  2,050,757 | MAPE:  249.83% | R²:  0.2857


In [57]:
print("\n" + "="*70)
print("🔮 MODEL 2: FACEBOOK PROPHET (With Regressors)")
print("="*70)

# Prepare data for Prophet
prophet_train = train_data[['Date', 'Sales', 'Promo', 'SchoolHoliday']].copy()
prophet_train.columns = ['ds', 'y', 'promo', 'school_holiday']

# Train Prophet
model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=15.0,
    seasonality_mode='multiplicative',
    interval_width=0.95
)
model.add_regressor('promo')
model.add_regressor('school_holiday')

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model.fit(prophet_train)

# Forecast
future = pd.DataFrame({
    'ds': test_data['Date'],
    'promo': test_data['Promo'].values,
    'school_holiday': test_data['SchoolHoliday'].values
})

prophet_forecast = model.predict(future)['yhat'].values
prophet_metrics = calculate_metrics(y_true, prophet_forecast, "Prophet")
results_list.append(('Prophet', prophet_metrics))


🔮 MODEL 2: FACEBOOK PROPHET (With Regressors)


01:02:31 - cmdstanpy - INFO - Chain [1] start processing
01:02:32 - cmdstanpy - INFO - Chain [1] done processing


✓ Prophet              → RMSE:    615,994 | MAE:    470,823 | MAPE:   24.99% | R²:  0.9568


In [58]:
print("\n" + "="*70)
print("🚀 MODEL 3: XGBOOST (ML with Features)")
print("="*70)

def prepare_xgb_features(df, lags=[1, 7, 14, 21]):
    """Create temporal + lag + rolling features"""
    df_model = df.copy()
    
    # Temporal
    df_model['day_of_week'] = df_model['Date'].dt.dayofweek
    df_model['month'] = df_model['Date'].dt.month
    df_model['quarter'] = df_model['Date'].dt.quarter
    df_model['week'] = df_model['Date'].dt.isocalendar().week
    df_model['is_weekend'] = (df_model['day_of_week'] >= 5).astype(int)
    
    # Lags
    for lag in lags:
        df_model[f'sales_lag_{lag}'] = df_model['Sales'].shift(lag)
        df_model[f'customers_lag_{lag}'] = df_model['Customers'].shift(lag)
    
    # Rolling
    for window in [7, 14]:
        df_model[f'sales_rolling_mean_{window}'] = df_model['Sales'].rolling(window).mean()
        df_model[f'sales_rolling_std_{window}'] = df_model['Sales'].rolling(window).std()
    
    # Business
    df_model['sales_per_customer'] = df_model['Sales'] / (df_model['Customers'] + 1)
    
    return df_model.dropna()

print("Preparing features...")
train_prepared = prepare_xgb_features(train_data)
test_prepared = prepare_xgb_features(test_data)

feature_cols = [col for col in train_prepared.columns if col not in ['Date', 'Sales']]
X_train = train_prepared[feature_cols]
y_train = train_prepared['Sales']
X_test = test_prepared[feature_cols]
y_test_xgb = test_prepared['Sales'].values

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train XGBoost
print("Training XGBoost...")
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train_scaled, y_train, verbose=False)

xgb_forecast = xgb_model.predict(X_test_scaled)

# Align lengths (feature engineering drops rows)
min_len = min(len(test_data), len(xgb_forecast))
xgb_forecast = xgb_forecast[:min_len]
y_true_xgb = test_data['Sales'].values[-min_len:]

xgb_metrics = calculate_metrics(y_true_xgb, xgb_forecast, "XGBoost")
results_list.append(('XGBoost', xgb_metrics))

# Feature importance
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False).head(15)

print("\n📊 Top 15 Features:")
print(importance_df)

# Save feature importance chart
fig = go.Figure(go.Bar(y=importance_df['Feature'], x=importance_df['Importance'], 
                       orientation='h', marker_color='skyblue'))
fig.update_layout(title='XGBoost - Top 15 Feature Importance', height=600)
save_plotly_chart(fig, 'M3_03_xgboost_features.html')


🚀 MODEL 3: XGBOOST (ML with Features)
Preparing features...
Training XGBoost...
✓ XGBoost              → RMSE:     64,389 | MAE:     53,351 | MAPE:    1.64% | R²:  0.9995

📊 Top 15 Features:
                  Feature  Importance
0               Customers    0.604569
21     sales_per_customer    0.280619
8              is_weekend    0.043440
3            StateHoliday    0.022373
1                   Promo    0.016485
4             day_of_week    0.008767
15           sales_lag_21    0.005330
19  sales_rolling_mean_14    0.002846
16       customers_lag_21    0.001826
9             sales_lag_1    0.001729
18    sales_rolling_std_7    0.001603
2           SchoolHoliday    0.001524
7                    week    0.001499
14       customers_lag_14    0.001488
20   sales_rolling_std_14    0.001223
✅ Chart saved: M3_03_xgboost_features.html


In [59]:
print("\n" + "="*70)
print("🌲 MODEL 4: RANDOM FOREST")
print("="*70)

print("Training Random Forest...")
rf_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_scaled, y_train)

rf_forecast = rf_model.predict(X_test_scaled)[:min_len]
rf_metrics = calculate_metrics(y_true_xgb, rf_forecast, "Random Forest")
results_list.append(('Random Forest', rf_metrics))


🌲 MODEL 4: RANDOM FOREST
Training Random Forest...
✓ Random Forest        → RMSE:    108,398 | MAE:     60,042 | MAPE:    1.33% | R²:  0.9987


In [60]:
print("\n" + "="*70)
print("🧠 MODEL 5: LSTM NEURAL NETWORK (Corrected)")
print("="*70)

def create_lstm_sequences(train_df, test_df, lookback=7):
    """Create proper LSTM sequences"""
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    feature_set = ['Sales', 'Customers', 'Promo']
    
    # Fit scalers on train
    scaler_X.fit(train_df[feature_set])
    scaler_y.fit(train_df[['Sales']])
    
    # Transform
    X_train_scaled = scaler_X.transform(train_df[feature_set])
    y_train_scaled = scaler_y.transform(train_df[['Sales']]).flatten()
    
    # Create sequences
    X_tr, y_tr = [], []
    for i in range(lookback, len(train_df)):
        X_tr.append(X_train_scaled[i-lookback:i])
        y_tr.append(y_train_scaled[i])
    
    X_tr = np.array(X_tr)
    y_tr = np.array(y_tr)
    
    # Test sequences
    X_test_scaled = scaler_X.transform(test_df[feature_set])
    combined = np.vstack([X_train_scaled[-lookback:], X_test_scaled])
    
    X_te = []
    for i in range(lookback, len(combined)):
        X_te.append(combined[i-lookback:i])
    
    X_te = np.array(X_te)
    y_test_actual = test_df['Sales'].values[:len(X_te)]
    
    return X_tr, y_tr, X_te, y_test_actual, scaler_y

print("Creating LSTM sequences...")
X_tr, y_tr, X_te, y_te_lstm, scaler_lstm = create_lstm_sequences(
    train_data, test_data, lookback=7
)

print(f"X_train: {X_tr.shape}, y_train: {y_tr.shape}")
print(f"X_test: {X_te.shape}, y_test: {len(y_te_lstm)}")

# Build LSTM
print("\nBuilding LSTM model...")
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(7, 3)),
    Dropout(0.2),
    LSTM(25),
    Dropout(0.2),
    Dense(12, activation='relu'),
    Dense(1)
])
lstm_model.compile(optimizer=Adam(0.001), loss='mse')

print("Training LSTM...")
lstm_model.fit(X_tr, y_tr, epochs=30, batch_size=64, validation_split=0.2, verbose=0)

# Predict
y_pred_scaled = lstm_model.predict(X_te, verbose=0).flatten()
y_lstm = scaler_lstm.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

lstm_metrics = calculate_metrics(y_te_lstm, y_lstm, "LSTM")
results_list.append(('LSTM', lstm_metrics))


🧠 MODEL 5: LSTM NEURAL NETWORK (Corrected)
Creating LSTM sequences...
X_train: (863, 7, 3), y_train: (863,)
X_test: (42, 7, 3), y_test: 42

Building LSTM model...
Training LSTM...
✓ LSTM                 → RMSE:    765,947 | MAE:    529,306 | MAPE:   38.31% | R²:  0.9332


In [61]:
print("\n" + "="*70)
print("🎭 ENSEMBLE MODEL (Weighted Voting)")
print("="*70)

# Align all forecasts to same length
min_ensemble_len = min(len(y_true_xgb), len(y_te_lstm))

ensemble_forecast = (
    0.40 * xgb_forecast[:min_ensemble_len] +
    0.30 * rf_forecast[:min_ensemble_len] +
    0.20 * prophet_forecast[-min_ensemble_len:] +
    0.10 * arima_forecast[-min_ensemble_len:]
)

y_true_ensemble = y_true_xgb[:min_ensemble_len]
ensemble_metrics = calculate_metrics(y_true_ensemble, ensemble_forecast, "Ensemble")
results_list.append(('Ensemble', ensemble_metrics))


🎭 ENSEMBLE MODEL (Weighted Voting)
✓ Ensemble             → RMSE:    307,100 | MAE:    261,871 | MAPE:   19.88% | R²:  0.9897


In [65]:
print("\n" + "="*70)
print("🏆 FINAL MODEL COMPARISON")
print("="*70 + "\n")

# Create results DataFrame with rounding
results_df = pd.DataFrame([
    {
        'Model': name,
        'RMSE': round(metrics.get('RMSE', np.nan), 0),
        'MAE': round(metrics.get('MAE', np.nan), 0),
        'MAPE': round(metrics.get('MAPE', np.nan), 2),
        'R2': round(metrics.get('R2', np.nan), 4)
    }
    for name, metrics in results_list
])

# Sort by RMSE ascending
results_df = results_df.sort_values('RMSE', ascending=True).reset_index(drop=True)

# Print table
print(results_df.to_string(index=False))

# Best model (lowest RMSE)
best_model_row = results_df.loc[results_df['RMSE'].idxmin()]
best_model_name = best_model_row['Model']
best_rmse = best_model_row['RMSE']
best_mape = best_model_row['MAPE']

print("\n" + "="*70)
print(f"🥇 BEST MODEL: {best_model_name}")
print(f"   RMSE: {best_rmse:,.0f}")
print(f"   MAPE: {best_mape:.2f}%")
print("="*70)

# Visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['RMSE Comparison', 'MAE Comparison', 'MAPE Comparison', 'Model Ranking']
)

fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['RMSE']), row=1, col=1)
fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['MAE']), row=1, col=2)
fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['MAPE']), row=2, col=1)
fig.add_trace(go.Bar(
    x=results_df['Model'],
    y=list(range(1, len(results_df) + 1))
), row=2, col=2)

fig.update_layout(
    height=800,
    title_text="📊 Model Performance Comparison",
    showlegend=False
)

save_plotly_chart(fig, 'M3_10_model_comparison.html')

# Save results
results_df.to_csv('data/milestone3_model_results.csv', index=False)
print("\n✅ Results saved: data/milestone3_model_results.csv")



🏆 FINAL MODEL COMPARISON

         Model      RMSE       MAE   MAPE      R2
       XGBoost   64389.0   53351.0   1.64  0.9995
 Random Forest  108398.0   60042.0   1.33  0.9987
      Ensemble  307100.0  261871.0  19.88  0.9897
       Prophet  615994.0  470823.0  24.99  0.9568
          LSTM  765947.0  529306.0  38.31  0.9332
Seasonal Naive 1651649.0 1094949.0  16.33  0.6893
ARIMA(2, 0, 3) 2504064.0 2050757.0 249.83  0.2857
Moving Average 3088281.0 2148412.0 458.57 -0.0864
         Naive 3442355.0 2425738.0 516.39 -0.3498

🥇 BEST MODEL: XGBoost
   RMSE: 64,389
   MAPE: 1.64%
✅ Chart saved: M3_10_model_comparison.html

✅ Results saved: data/milestone3_model_results.csv


In [66]:
print("\n" + "="*70)
print("📊 RESIDUAL ANALYSIS (Best Model)")
print("="*70)

# Get best model's predictions
if best_model_name == 'XGBoost':
    best_predictions = xgb_forecast[:min_ensemble_len]
    y_true_best = y_true_xgb[:min_ensemble_len]
elif best_model_name == 'Random Forest':
    best_predictions = rf_forecast[:min_ensemble_len]
    y_true_best = y_true_xgb[:min_ensemble_len]
elif best_model_name == 'Ensemble':
    best_predictions = ensemble_forecast
    y_true_best = y_true_ensemble
else:
    best_predictions = arima_forecast[-min_ensemble_len:]
    y_true_best = y_true_ensemble

residuals = y_true_best - best_predictions

print(f"Mean residual: {np.mean(residuals):.2f} (should be ~0)")
print(f"Std residual: {np.std(residuals):.2f}")
print(f"Min residual: {np.min(residuals):,.0f}")
print(f"Max residual: {np.max(residuals):,.0f}")


📊 RESIDUAL ANALYSIS (Best Model)
Mean residual: 10611.38 (should be ~0)
Std residual: 63508.17
Min residual: -145,049
Max residual: 121,447


In [67]:
fig = make_subplots(rows=2, cols=2,
    subplot_titles=['Residuals Over Time', 'Distribution', 'Predicted vs Actual', 'Residuals vs Predicted'])

# Time series
fig.add_trace(go.Scatter(y=residuals, mode='markers+lines', name='Residuals'), row=1, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="red", row=1, col=1)

# Histogram
fig.add_trace(go.Histogram(x=residuals, nbinsx=30, name='Distribution'), row=1, col=2)

# Predicted vs Actual
fig.add_trace(go.Scatter(y=y_true_best, mode='lines', name='Actual', line=dict(color='green')), row=2, col=1)
fig.add_trace(go.Scatter(y=best_predictions, mode='lines', name='Predicted', line=dict(color='red')), row=2, col=1)

# Residuals vs Predicted
fig.add_trace(go.Scatter(x=best_predictions, y=residuals, mode='markers', name='Res vs Pred'), row=2, col=2)
fig.add_hline(y=0, line_dash="dash", line_color="red", row=2, col=2)

fig.update_layout(height=800, title_text=f"Residual Analysis - {best_model_name}", showlegend=False)
save_plotly_chart(fig, 'M3_11_residual_analysis.html')

✅ Chart saved: M3_11_residual_analysis.html


In [68]:
print("\n" + "="*70)
print("📈 FINAL FORECAST VISUALIZATION")
print("="*70)

fig = go.Figure()

# Training data
fig.add_trace(go.Scatter(x=train_data['Date'], y=train_data['Sales'],
    mode='lines', name='Training Data', line=dict(color='blue', width=1)))

# Test data actual
fig.add_trace(go.Scatter(x=test_data['Date'][-len(y_true_best):], y=y_true_best,
    mode='lines', name='Actual Test Data', line=dict(color='green', width=2)))

# Best forecast
test_dates = test_data['Date'].values[-len(best_predictions):]
fig.add_trace(go.Scatter(x=test_dates, y=best_predictions,
    mode='lines', name=f'{best_model_name} Forecast', 
    line=dict(color='red', width=2, dash='dash')))

# Confidence interval
std_residual = np.std(residuals)
fig.add_trace(go.Scatter(x=test_dates, y=best_predictions + std_residual,
    fill=None, mode='lines', line_color='rgba(0,0,0,0)', showlegend=False))
fig.add_trace(go.Scatter(x=test_dates, y=best_predictions - std_residual,
    fill='tonexty', mode='lines', line_color='rgba(0,0,0,0)',
    name='±1 Std Dev', fillcolor='rgba(255,0,0,0.2)'))

fig.update_layout(
    title=f'🏆 Final Forecast: {best_model_name} (MAPE: {best_mape:.2f}%)',
    xaxis_title='Date',
    yaxis_title='Sales',
    height=600,
    hovermode='x unified'
)

save_plotly_chart(fig, 'M3_12_final_forecast.html')


📈 FINAL FORECAST VISUALIZATION
✅ Chart saved: M3_12_final_forecast.html


In [70]:
import joblib
from datetime import datetime

print("\n" + "="*70)
print("💾 SAVING MODELS & RESULTS")
print("="*70)

# Save best model
if 'xgb_model' in globals() and best_model_name == 'XGBoost':
    joblib.dump(xgb_model, 'models/best_model.pkl')
    print("✅ XGBoost model saved")
elif 'rf_model' in globals() and best_model_name == 'Random Forest':
    joblib.dump(rf_model, 'models/best_model.pkl')
    print("✅ Random Forest model saved")

# Save predictions
predictions_export = pd.DataFrame({
    'Date': test_data['Date'].values[-len(y_true_best):],
    'Actual': y_true_best,
    'Predicted': best_predictions,
    'Residual': residuals,
    'Error_Percent': (np.abs(residuals) / y_true_best * 100)
})
predictions_export.to_csv('data/final_predictions.csv', index=False)
print("✅ Predictions exported: data/final_predictions.csv")


💾 SAVING MODELS & RESULTS
✅ XGBoost model saved
✅ Predictions exported: data/final_predictions.csv


In [71]:
summary = {
    'best_model': best_model_name,
    'rmse': float(best_rmse),
    'mae': float(results_df.iloc[best_idx]['MAE']),
    'mape': float(best_mape),
    'r2': float(results_df.iloc[best_idx]['R2']),
    'timestamp': datetime.now().isoformat(),
    'all_results': results_df.to_dict()
}

import json
with open('data/milestone3_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
    
print("✅ Summary saved: data/milestone3_summary.json")
print(f"\n🎉 MILESTONE 3 COMPLETE!")
print(f"   Best Model: {best_model_name}")
print(f"   RMSE: {best_rmse:,.0f}")
print(f"   MAPE: {best_mape:.2f}%")

✅ Summary saved: data/milestone3_summary.json

🎉 MILESTONE 3 COMPLETE!
   Best Model: XGBoost
   RMSE: 64,389
   MAPE: 1.64%
